## Crassus: Development notes
---

In [13]:
# Python imports
from Bio import SeqIO
import os, glob

### Reference crAss set

Originally I used all the genomes that we have (~2k), comprising complete and uncomplete genomes. Besides this might be too much redundancy, classification by ANI would be incorrect for uncomplete genomes. Because of this **I decide to use only the ones listed at `contigs_clustering.xlsx`, with the complete and taxonomically classified genomes from the proposal**. Using type species would be too few though. 

So, I change the whole reference set to include in the the container. 846 complete crAss genomes (excluding crass-env) in `contigs_clustering.xlsx`, but I still add the reference Bas' crass (crass001 is already in the list) and the two outgroups, so it makes 849 at the end. I change the .fasta files under `/genomes` and the proteins in `all_crass_proteins.faa` according to this. ⚠️ **WARNING** ⚠️ Because of the lack/split of TerL gene in 8 genomes, the final number is 841, read below.

Regarding the terminases, I extract them with Biopython. I grab all the trimmed, merged terminases from `bas/february_2021/terLterL_sequences_trimmed_merged.faa`:

In [20]:
# read "crassus_genomes.txt" with the target genomes ids
target_genomes = [line.strip() for line in open("crassus_genomes.txt").readlines()]

# read the .faa file with all the terminases. Store the ones from the target genomes
to_write = list()
records = SeqIO.parse("container/terL_sequences_trimmed_merged.faa", "fasta")
for record in records:
    genome = record.id.split("|")[0]
    if genome in target_genomes:
        to_write.append(record)
        target_genomes.remove(genome)
        
with open("container/crass_reference_TerL.faa", "w") as fout:
    SeqIO.write(to_write, fout, "fasta")
                
# print genomes without a final TerL protein 
print(target_genomes)
len(target_genomes)

['3300008523_____Ga0115231_1000095', 'OAWB01000083', '3300014553_____Ga0134451_100057', 'ERR589831_NODE_127_length_97341_cov_28.7607', 'OLOZ01000093', 'OLQU01000060', 'ERR589503_NODE_260_length_95985_cov_44.2112', 'ERR589804_NODE_183_length_96082_cov_14.9851']


8

Note well the 8 genomes without a TerL gene. **I remove them from the reference set so far**. 

In [21]:
# remove them from the genomes folder
for genome in target_genomes:
    if os.path.isfile(f"container/genomes/{genome}.fasta"):
        print(f"deleting {genome} genome")
        os.remove(f"container/genomes/{genome}.fasta")
        
# remove them from `all_crass_proteins.faa` 
to_write = list()
records = SeqIO.parse("container/all_crass_proteins.faa", "fasta")
for record in records:
    genome = record.id.split("|")[0]
    if genome not in target_genomes:
        to_write.append(record)

with open("container/all_crass_proteins.faa", "w") as fout:
    SeqIO.write(to_write, fout, "fasta")

---
### TerL tree

The reference set are 841 genomes with their proteins and terminases. I don't know how long is gonna take to align all these sequences + the sequences found by the pipeline.

`mafft-einsi` took 15 mins with 841 seqs and 93 CPUs.
- `(mafft_env) danielc@mutant5:/linuxhome/tmp/danielc/crassus_terl:time mafft-einsi --thread 93 --quiet crass_reference_TerL.faa > crass_reference_TerL.mafft-einsi`

---
### Branch and merge

I want to play with this. Let's refactor de pipeline, brach **refactor**. 

\# create branch and move to it\
`$ git checkout -b refactor`

\# change the stuff I want to change and commit it\
`$ git add .gitignore dev_notes.ipynb workflow/Snakefile workflow/rules/`
`$ git commit -m "refactor initial rules"`